In [ ]:
import os
import sys
import glob
import random
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt 
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
from sklearn.metrics import accuracy_score

from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50

import numpy as np
% matplotlib inline


# Import modules every time you run code imported using %aimport
%load_ext autoreload
%autoreload 1

# Add the src directory for functions
src_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'src')
print(src_dir)
sys.path.append(src_dir)

# import my functions:
%aimport models
from models import*
%aimport functions
from functions import*

# Base Directory where data is stored
base_data_dir = '/home/rbbidart/project/rbbidart/breakHis/'
base_out_dir = '/home/rbbidart/breakHis_out'

In [ ]:
def visualize_incp(model, img_loc):
    image_shape = (512, 512)
    img = Image.open(img_loc)
    img = np.array(img.resize(image_shape))
    
    get_output = K.function([model.layers[0].input, K.learning_phase()],
                                  [model.layers[3].output])
    
    class_weights = model.layers[-1].get_weights()[0]

    pred_model = Model(inputs=model.input, outputs=model.get_layer(index=-1).output)
    cov_model = Model(inputs=model.input, outputs=model.get_layer(index=310).output)
     
    predictions = pred_model.predict(np.expand_dims(img, axis=0))
    conv_outputs = cov_model.predict(np.expand_dims(img, axis=0))
    conv_outputs = conv_outputs[0, :, :, :]
        
    cam = np.zeros(dtype = np.float32, shape = [class_weights.shape[1], conv_outputs.shape[0], conv_outputs.shape[0]])
    print('conv_outputs.shape', conv_outputs.shape)
    print('class_weights.shape', class_weights.shape)
    print('cam.shape', cam.shape)

    
    #Create the class activation map.
    # Convert final conv layer from features (2048 dim) to predictions (8)
    # Already learned mapping after global average pooling, so just appply it to every spatial location
    
    for target_class in range(class_weights.shape[1]):
        for i, w in enumerate(class_weights[:, target_class]):
            cam[target_class, :, :] += w * conv_outputs[:, :, i]
            
    
    print('cam.shape', cam.shape )
    plt.imshow(cam[0, :, :])
    plt.figure(figsize=(10,6))
    plt.show()
    
    plt.imshow(cam[1, :, :])
    plt.figure(figsize=(10,6))
    plt.show()
    
    plt.imshow(cam[2, :, :])
    plt.figure(figsize=(10,6))
    plt.show()
    
    plt.figure(figsize=(10,6))
    plt.imshow(img)
    plt.show()


In [ ]:
model_loc = os.path.join(base_out_dir, 'inception_pre_197_simple', 'fine_tune_.14-0.77.hdf5')
# model = load_model(model_loc)

# img_loc = '/home/rbbidart/project/rbbidart/breakHis/mkfold_keras_8/fold1/100/valid/B_F/SOB_B_F-14-14134-100-001.png'
img_loc = '/home/rbbidart/project/rbbidart/breakHis/mkfold_keras_8/fold1/100/valid/B_F/SOB_B_F-14-14134-100-021.png'

visualize_incp(model, img_loc)